Loads the file names into a dict and pickles...

http://nptdms.readthedocs.io/en/latest/

In [2]:
data_dir = ".\\data\\"

#filename = "2018-06-20-10-44-11.tdms"
#keyence_filename= "2018-06-20-10-44-18-KEYENCE.tdms"

# SLC-2445, air, slow
#filename = "2018-07-12-18-36-41.tdms"
#keyence_filename= "2018-07-12-18-36-56-KEYENCE.tdms"

# SLC-2445, air, fast
#filename = "2018-07-12-18-51-08.tdms"
#keyence_filename= "2018-07-12-18-51-17-KEYENCE.tdms"

#filename = "2018-07-17-10-17-29.tdms"  # SLC-2445, vac, fast
#filename = "2018-07-17-09-45-52.tdms" # SLC-2445, vac, slow
#keyence_filename = None

# SLC-1730, air, slow
#filename = "2018-07-25-10-58-32.tdms"
#keyence_filename= "2018-07-25-10-58-24-KEYENCE.tdms"

# SLC-1730, air, fast
#filename = "2018-07-25-11-13-44.tdms"
#keyence_filename= "2018-07-25-11-13-50-KEYENCE.tdms"

# SLC-1730, air, fast
#filename = "2018-07-26-10-30-38.tdms"
#keyence_filename= "2018-07-26-10-31-06-KEYENCE.tdms"

# SLC-1730, air, slow
#filename = "2018-07-26-10-44-20.tdms"
#keyence_filename= "2018-07-26-10-44-40-KEYENCE.tdms"

# SLC-1730, vac
#filename = "2018-07-26-14-39-24.tdms"
#keyence_filename= None

# SLC-2445, vac, 72hrs
#filename = "2018-07-27-14-45-18.tdms"
#keyence_filename = None

# SLC-1730, vac, 48hrs
filename = "2018-07-30-16-19-44.tdms"
keyence_filename = None

# SLC-2445, air, 30hrs, keyence
filename = "2018-08-30-10-01-54.tdms"
keyence_filename = "2018-08-30-10-03-14-KEYENCE.tdms"

# SLC-1730, air, 30hrs??, keyence
#filename = "2018-08-31-15-46-55.tdms"
#keyence_filename = "2018-08-31-15-46-59-KEYENCE.tdms"

#duty test SLC-1730
filename = "2018-09-27-09-10-21.tdms"
keyence_filename = None


In [3]:
#Import SmarAct/Temperature data
from nptdms import TdmsFile

# Load the file
tdms_file = TdmsFile(data_dir + filename)

props = tdms_file.object().properties
props['Acceleration'] = props['Acceleration']/1e9
props['Velocity'] = props['Velocity']/1e9
props['SetPoint1'] = props['SetPoint1']/1e9
props['SetPoint2'] = props['SetPoint2']/1e9

try:
    if props['Note'] == "Note":
        del props['Note']
    props['Note'] = ", ".join(props['Note'].split("\n"))
except KeyError:
    print("Enter a note for this data:  ")
    props['Note'] = input()
    if props['Note'] is "":
        props['Note'] = "N/A"
        

info = \
"""Filename: \t{name}
Dwell time: \t{Dwell}s
Velocity: \t{Velocity}mm/s
Acceleration: \t{Acceleration}mm/s²
Max freq: \t{Freq}Hz
Setpoints: \t{SetPoint1}mm, {SetPoint2}mm
Notes: \t{Note}""".format(**props)

print(info)

info_key, info_val = zip(*[i.split('\t') for i in info.split('\n')])

# Read the setpoint data
set_point = dict(
    time  = tdms_file.object('SetPoint', 'Time').data,
    data  = tdms_file.object('SetPoint', 'SET_PT').data
    )

# Read the position data
position = dict(
    time  = tdms_file.object('Position', 'Time').data,
    data  = tdms_file.object('Position', 'SM-POS').data
    )

# Convert pm into mm
set_point['data'] = [d/1e9 for d in set_point['data']]
position['data']  = [d/1e9 for d in position['data']]

# Read the temperature data
temperature = dict(
    time  = tdms_file.object('Temperature', 'Time').data,
    stage = tdms_file.object('Temperature', 'T_STAGE').data,
    base  = tdms_file.object('Temperature', 'T_BASE').data,
    amb   = tdms_file.object('Temperature', 'T_AMB').data
    )

ValueError: Zero channel data size but non-zero data length based on segment offset.

In [ ]:
# Import Keyence data
from nptdms import TdmsFile

if keyence_filename is not None:
    # Load the file
    keyence_tdms_file = TdmsFile(data_dir + keyence_filename)

    keyence = dict(
        time  = keyence_tdms_file.object('DATA', 'Time').data,
        # The keyence data is -ve - invert it!
        data1 = keyence_tdms_file.object('DATA', 'OUT1').data * -1
    )
    
    try:
        keyence['data2'] = keyence_tdms_file.object('DATA', 'OUT2').data * -1
    except KeyError as e:
        print(e)

In [ ]:
full_data = dict(
    set_point   = set_point,
    position    = position,
    temperature = temperature,
    info        = dict([i.split('\t') for i in info.split('\n')])
)

if keyence_filename is not None:
    full_data['keyence'] = keyence

In [ ]:
print("Full dataset:")
for k in full_data:
    print("{}: \t{} points".format(k, len(full_data[k][[*full_data[k]][0]])))

In [ ]:
for k in full_data:
    print(k)
    for l in full_data[k]:
        print("\t"+l)

In [ ]:
# Dump the data to file...

import pickle

if keyence_filename is not None:
    pickle_file = data_dir + "_".join([filename, keyence_filename, "full_data.p"])
else:
    pickle_file = data_dir + "_".join([filename, "full_data.p"])

pickle.dump(full_data, open(pickle_file, 'wb'))

print("Full dataset stored in {}".format(pickle_file))

In [ ]:
del full_data